# Med Student Qualtrics Results - Usability Questionaire
-----
The clinical questionaire is comprised of 3 tasks (Diabetes, Hypertension, Depression) and 21 questions. There are several ways to begin analyzing the data set:

- [SUS Composite Scoring Difference](#sus)
- [NASA-TLX Composite Scoring Difference](#nasa)
- [Individual-Question Comparisons](#iqc)

<a id='sus'></a>
### SUS Composite Scoring Difference
------
to-be-completed

<a id='nasa'></a>
### NASA-TLX Composite Scoring Difference
-------
to-be-completed

<a id='iqc'></a>
### Individual-Question Comparisons
--------
There are 3 tasks (Ease of Use, Cognitive Workload, and User Satisfaction) and 18 questions:

- Ease of Use
    - [I think that I would like to use this application frequently.](#e1)
    - [I found the application unnecessarily complex.](#e2)
    - [I thought the application was easy to use.](#e3)
    - [I think that I would need the support of a technical person](#e4)
    - [I found the various functions in the application were well integrated.](#e5)
    - [I thought there was too much inconsistency in this application.](#e6)
    - [I would imagine that most people would learn to use this application very quickly.](#e7)
    - [I found the application very cumbersome to use.](#e8)
    - [I felt very confident using the application.](#e9)
    - [I needed to learn a lot of things before I could get going with this application.](#e10)
- Cognitive Workload
    - [How mentally demanding was the task?](#c1)
    - [How much time pressure did you feel?](#c2)
    - [How successful were you in accomplishing what you were asked to do?](#c3)
    - [How hard did you have to work to accomplish your level of performance?](#c4)
    - [How insecure, discouraged, irritated, stressed, and annoyed were you?](#c5)
- Satisfaction
    - [I believe this medication display would help improve my ability](#s1)
    - [I believe this medication display would aid me in providing a better treatment](#s2)
    - [I believe this medication display would improve patient outcomes.](#s3)